#1)-Importing Dependencies

In [1]:
# Data handling and traditional algebraic operations
import numpy as np
import pandas as pd
# ML dependencies
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
# DL dependencies
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset as TorchDataset

import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
from nltk import pos_tag
nltk.download('punkt')

# Text manipulation tools
import re
import string
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#2)-Data Preprocessing

##2.1-Data importing

First, we are going to use two standard datasets for grammatical error detection:

*   Lang-8
*   NUS Social Media Text Normalization and Translation Corpus.



###Lang-8

In [18]:
# Lang-8 loading and extraction of correct and incorrect sentences
path = "/content/drive/MyDrive/Colab Notebooks/2)-Machine Learning/4)-NeuroMatch Academy -DL/NLP - Project/Data/entries.train"
f1 = open(path)
lines1 = f1.readlines()
inp1 = [] # list for incorrect sentences
tgt1 = [] # storing

for i in lines1:
    lst = i.split("\t")

# IF LENGTH OF THE LIST IS GREATER THAN 5 THEN CORRECT SENTTENCE EXISTS OTHERWISE ONLY INCORRECT SENTENCE IS PRESENT
    if len(lst)>5  :     #IF LENGTH IS GREATER THAN 5
        inp1.append(lst[-2]) # APPEND SECONG LAST ITEM IN LIST WHICH IS INCORRECT SENTENCE
        tgt1.append(lst[-1]) # APPEND LAST ITEM IN THE LIST WHICH IS

In [19]:
df = pd.DataFrame()
df['y'] = list('1'*len(inp1))
df['input'] = inp1
df['output'] = tgt1


In [20]:
df.head()

,y,input,output
0,1,And he took in my favorite subject like soccer .,And he took in my favorite subjects like socce...
1,1,"Actually , who let me know about Lang - 8 was ...","Actually , he was the one who let me know abou..."
2,1,His Kanji 's ability is much better than me .,His Kanji ability is much better than mine .\n
3,1,"We 've known each other for only half a year ,...","We 've known each other for only half a year ,..."
4,1,I heard a sentence last night when I watched TV .,I heard a sentence last night when I was watch...


In [21]:
df.describe()

,y,input,output
count,509163,509163,509163
unique,1,504476,500801
top,1,thank you .,Hello !\n
freq,509163,40,90


###NUS Social Media Text Normalization and Translation Corpus

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/2)-Machine Learning/4)-NeuroMatch Academy -DL/NLP - Project/Data/10gec_annotations/en2cn-2k.en2nen2cn"
f2 = open(path,"r",encoding="UTF-8")


#f2 = open("/content/drive/MyDrive/Colab Notebooks/cs2/text_sms/release/en2cn-2k.en2nen2cn","r",encoding="UTF-8") # READING THE FILE

lines2 = f2.readlines() # STORING ALL THE LINES IN A VARIABLE
inp2 = [] # LIST FOR STORING INCORRECT SENTENCES
tgt2 = [] # LIST FOR STORING CORRECT SENTENCES

# THE DASET CONTAINS 2000 DATAPOINTS, THEREFORE RUNNING THE LOOP FOR 2000 TIMES
for i in range(2000):
    inp2.append(lines2[i*3]) #APPEDING FIRST ROW FOR EACH DATAPOINT
    tgt2.append(lines2[i*3+1]) # APPENDING SECOND ROW FOR EACH DATAPOINT

FileNotFoundError: ignored

##2.2-Data cleaning

###2.2.1-Data formating

In [15]:
def remove_spaces(text):
    text = re.sub(r" '(\w)",r"'\1",text)
    text = re.sub(r" \,",",",text)
    text = re.sub(r" \.+",".",text)
    text = re.sub(r" \!+","!",text)
    text = re.sub(r" \?+","?",text)
    text = re.sub(" n't","n't",text)
    text = re.sub("[\(\)\;\_\^\`\/]","",text)
    return text

def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

def preprocess(text):
    text = re.sub("\n","",text)
    text = remove_spaces(text)
    text = re.sub(r"\.+",".",text)
    text = re.sub(r"\!+","!",text)
    text = decontract(text)
    text = re.sub("[^A-Za-z0-9 ]+","",text)
    text = text.lower()
    return text

In [22]:
df["enc_input"] = df.input.apply(preprocess)
df["dec_input"] = df.input.apply(preprocess)
df["dec_output"] = df.output.apply(preprocess)
df =df.drop(["input","output"],axis=1)
df = df[df.enc_input.notnull()]
df = df[df.dec_input.notnull()]
df = df[df.dec_output.notnull()]
df = df.drop_duplicates()

In [ ]:
df.head()

###2.2.2-Removing null values and duplicates

In [10]:
df.shape

(503901, 4)

In [11]:
df = df.drop_duplicates()
df = df[df.enc_input.notnull()]
df = df[df.dec_input.notnull()]
df = df[df.dec_output.notnull()]
print(df.shape)
df.head()

(503901, 4)


,y,enc_input,dec_input,dec_output
0,1,and he took in my favorite subject like soccer,and he took in my favorite subject like soccer,and he took in my favorite subjects like soccer
1,1,actually who let me know about lang 8 was him,actually who let me know about lang 8 was him,actually he was the one who let me know about ...
2,1,his kanji is ability is much better than me,his kanji is ability is much better than me,his kanji ability is much better than mine
3,1,we have known each other for only half a year ...,we have known each other for only half a year ...,we have known each other for only half a year ...
4,1,i heard a sentence last night when i watched tv,i heard a sentence last night when i watched tv,i heard a sentence last night when i was watch...


##2.2-Tokenization

We crate a tokenizer that takes into account the special ``<start>`` and ``<end>`` tokens. So we first add these tokens to the data to ensure correct formating.

In [ ]:
# Adding <start> and <end> token
df["dec_input"] = "<start>"+ df["dec_input"]
df["dec_output"].iloc[0] = df["dec_output"].iloc[0] + "<end>"
df.head()

,y,enc_input,dec_input,dec_output
0,1,and he took in my favorite subject like soccer,<start>and he took in my favorite subject like...,and he took in my favorite subjects like socce...
1,1,actually who let me know about lang 8 was him,<start>actually who let me know about lang 8 ...,actually he was the one who let me know about ...
2,1,his kanji is ability is much better than me,<start>his kanji is ability is much better tha...,his kanji ability is much better than mine
3,1,we have known each other for only half a year ...,<start>we have known each other for only half ...,we have known each other for only half a year ...
4,1,i heard a sentence last night when i watched tv,<start>i heard a sentence last night when i wa...,i heard a sentence last night when i was watch...


We then split the original dataset into training, validation and test sets.

In [ ]:
# Train, validation and test split
df_train ,df_val_train = train_test_split(df, test_size=0.3,random_state = 16, stratify = df.y )
df_val, df_test = train_test_split(df_val_train, test_size=0.5, random_state = 16, stratify = df_val_train.y)

In [ ]:
print("Train Shape =",df_train.shape)
print("Val Shape =",df_val.shape)
print("Test Shape =",df_test.shape)

Train Shape = (352730, 4)
Val Shape = (75585, 4)
Test Shape = (75586, 4)


Finally, we run the tokenizer using ```get_tokenizer``, perform a word counter for future uses using the ``Counter()`` cosntructor and generate the input and output training vocabulary for the encoder using ```build_vocab_from_iterator``.

In [ ]:
from collections import Counter

def count_token_occurrences(tokens_list):
  counter = Counter()
  for sentence_tokens in tokens_list:
    counter.update(sentence_tokens)
  return counter

# Define the tokenizer function (use basic_english tokenizer)
tokenizer = get_tokenizer('basic_english')
traindata_in = df_train.dec_input.apply(str).tolist()
traindata_out = df_train.dec_output.apply(str).tolist()

# Tokenization and preprocessing for encoder input
enc_input_tokens = [tokenizer(sentence) for sentence in traindata_in]

# Tokenization and preprocessing for decoder input
dec_input_tokens = [tokenizer(sentence) for sentence in traindata_out]

# Build vocabulary for encoder input
counter_enc = count_token_occurrences(enc_input_tokens)
tk_inp = build_vocab_from_iterator(enc_input_tokens,specials=['<pad>'])
# Build vocabulary for decoder input
counter_dec = count_token_occurrences(dec_input_tokens)
tk_out = build_vocab_from_iterator(dec_input_tokens, specials=['<pad>', '<start>', '<end>'])


In [ ]:
print(type(tk_out.get_stoi()))

<class 'dict'>


###3.3-Text data into integer sequences

We now try to convert the text data into integer sequences wich also has a padding. This padding of sequences is necessary to ensure that all sequences in a batch have the same length. Padding adds special tokens (pad token) to the sequences to that all sequences pocesses the same amount of tokens.

In [ ]:
class Dataloader(DataLoader):
  def __init__(self, batch_size, dataset):
    self.dataset = dataset
    self.batch_size = batch_size
    self.total_points = self.dataset.encoder_in.shape[0]

  def __iter__(self):
    for i in range(len(self)):
      yield self.dataset[i]

  def __getitem__(self,i):
    start = i * self.batch_size
    stop = (i+1) * self.batch_size

    batch_enc = []
    batch_dec_input = []
    batch_dec_out = []

    for j in range(start, stop):
      a, b, c = self.dataset[j]
      batch_enc.append(a[0])
      batch_dec_input.append(b[0])
      batch_dec_out.append(c[0])

    batch_enc = torch.tensor(batch_enc)
    batch_dec_input = torch.tensor(batch_dec_input)
    batch_dec_out = torch.tensor(batch_dec_out)

    return [batch_enc, batch_dec_input], batch_dec_out

  def __len__(self):
    return int(self.total_points / self.batch_size)

In [ ]:
class conv_dataset(TorchDataset):
  def __init__(self, data, tk_inp, tk_out, max_len):
    self.encoder_in = data["enc_input"].apply(str).values
    self.decoder_in = data["dec_input"].apply(str).values
    self.decoder_out = data["dec_output"].apply(str).values
    self.tk_inp = tk_inp.get_stoi()
    self.tk_out = tk_out.get_stoi()
    self.tokenizer =  get_tokenizer('basic_english')
    self.max_len = max_len # max lengh of the sequences

  def __getitem__(self,i):
    # Input sequences
    encoder_seq = self.encoder_in[i]
    encoder_tokens = self.tokenizer(encoder_seq)
    encoder_indices = [self.tk_inp.vocab.stoi[token] for token in encoder_tokens]
    encoder_tensor = torch.tensor(encoder_indices)
    # Input encoder sequences
    decoder_in_seq = self.decoder_in[i]
    decoder_in_tokens = self.tokenizer(decoder_in_seq)
    decoder_in_indices = [self.tk_out.vocab.stoi[token] for token in decoder_in_tokens]
    decoder_in_tensor = torch.tensor(decoder_in_indices)
    # Input decoder sequences
    decoder_out_seq = self.decoder_out[i]
    decoder_tokens = self.tokenizer(decoder_out_seq)
    decoder_out_indices = [self.tk_out.vocab.stoi[token] for token in decoder_tokens]
    decoder_out_tensor = torch.tensor(decoder_out_indices)
    # Tokenizer padding
    encoder_tensor = F.pad(encoder_tensor, pad=(0, self.max_len - len(encoder_tensor)))
    decoder_in_tensor = F.pad(decoder_in_tensor, pad=(0, self.max_len - len (decoder_in_tensor)))
    decoder_out_tensor = F.pad(decoder_in_tensor, pad=(0, self.max_len - len (decoder_in_tensor)))

    return encoder_tensor, decoder_in_tensor, decoder_out_tensor

  def __len__(self):
    return len(self.encoder_in)



In [ ]:
class conv_dataset(TorchDataset):
  def __init__(self, data, tk_inp, tk_out, max_len):
    self.encoder_in = data["enc_input"].apply(str).values
    self.decoder_in = data["dec_input"].apply(str).values
    self.decoder_out = data["dec_output"].apply(str).values
    self.tk_inp = tk_inp.get_stoi()
    self.tk_out = tk_out.get_stoi()
    self.tokenizer =  get_tokenizer('basic_english')
    self.max_len = max_len # max lengh of the sequences

  def __getitem__(self,i):
    # Input sequences
    encoder_seq = self.encoder_in[i]
    encoder_tokens = self.tokenizer(encoder_seq)
    encoder_indices = [self.tk_inp[token] for token in encoder_tokens]
    encoder_tensor = torch.tensor(encoder_indices)
    # Input encoder sequences
    decoder_in_seq = self.decoder_in[i]
    decoder_in_tokens = self.tokenizer(decoder_in_seq)
    decoder_in_indices = [self.tk_out[token] for token in decoder_in_tokens]
    decoder_in_tensor = torch.tensor(decoder_in_indices)
    # Input decoder sequences
    decoder_out_seq = self.decoder_out[i]
    decoder_out_tokens = self.tokenizer(decoder_out_seq)
    decoder_out_indices = [self.tk_out[token] for token in decoder_out_tokens]
    decoder_out_tensor = torch.tensor(decoder_out_indices)
    # Tokenizer padding
    encoder_tensor = F.pad(encoder_tensor, pad=(0, self.max_len - len(encoder_tensor)))
    decoder_in_tensor = F.pad(decoder_in_tensor, pad=(0, self.max_len - len (decoder_in_tensor)))
    decoder_out_tensor = F.pad(decoder_in_tensor, pad=(0, self.max_len - len (decoder_in_tensor)))

    return encoder_tensor, decoder_in_tensor, decoder_out_tensor

  def __len__(self):
    return len(self.encoder_in)

In [ ]:
class conv_dataset(TorchDataset):
  def __init__(self, data, tk_inp, tk_out, max_len):
    self.encoder_in = data["enc_input"].apply(str).values
    self.decoder_in = data["dec_input"].apply(str).values
    self.decoder_out = data["dec_output"].apply(str).values
    self.tk_inp = tk_inp
    self.tk_out = tk_out
    self.tokenizer =  get_tokenizer('basic_english')
    self.max_len = max_len # max lengh of the sequences

  def __getitem__(self,i):
    # Input sequences
    encoder_seq = self.encoder_in[i]

    return encoder_seq

  def __len__(self):
    return len(self.encoder_in)

In [ ]:
import torch
from torch.utils.data import Dataset as TorchDataset
from torchtext.data.utils import get_tokenizer
import torch.nn.functional as F

class conv_dataset(TorchDataset):
    def __init__(self, data, tk_inp, tk_out, max_len):
        self.encoder_in = data["enc_input"].apply(str).values
        self.decoder_in = data["dec_input"].apply(str).values
        self.decoder_out = data["dec_output"].apply(str).values
        self.tk_inp = tk_inp.get_stoi()
        self.tk_out = tk_out.get_stoi()
        self.tokenizer = get_tokenizer('basic_english')
        self.max_len = max_len

    def __getitem__(self, i):
        # Input sequences
        encoder_seq = self.encoder_in[i]
        encoder_tokens = self.tokenizer(encoder_seq)
        encoder_indices = [self.tk_inp[token] for token in encoder_tokens]
        encoder_tensor = torch.tensor(encoder_indices)

        # Input encoder sequences
        decoder_in_seq = self.decoder_in[i]
        decoder_in_tokens = self.tokenizer(decoder_in_seq)
        # Special handling for <start> and <end> tokens
        decoder_in_indices = [self.tk_out['<start>']] + [self.tk_out[token] for token in decoder_in_tokens] + [self.tk_out['<end>']]
        decoder_in_tensor = torch.tensor(decoder_in_indices)

        # Input decoder sequences
        decoder_out_seq = self.decoder_out[i]
        decoder_tokens = self.tokenizer(decoder_out_seq)
        # Special handling for <start> and <end> tokens
        decoder_out_indices = [self.tk_out['<start>']] + [self.tk_out[token] for token in decoder_tokens] + [self.tk_out['<end>']]
        decoder_out_tensor = torch.tensor(decoder_out_indices)

        # Tokenizer padding
        encoder_tensor = F.pad(encoder_tensor, pad=(0, self.max_len - len(encoder_tensor)))
        decoder_in_tensor = F.pad(decoder_in_tensor, pad=(0, self.max_len - len(decoder_in_tensor)))
        decoder_out_tensor = F.pad(decoder_out_tensor, pad=(0, self.max_len - len(decoder_out_tensor)))

        return encoder_tensor, decoder_in_tensor, decoder_out_tensor

    def __len__(self):
        return len(self.encoder_in)


In [ ]:
import torch
from torch.utils.data import Dataset as TorchDataset
from torchtext.data.utils import get_tokenizer
import torch.nn.functional as F

class conv_dataset(TorchDataset):
    def __init__(self, data, tk_inp, tk_out, max_len):
        self.encoder_in = data["enc_input"].apply(str).values
        self.decoder_in = data["dec_input"].apply(str).values
        self.decoder_out = data["dec_output"].apply(str).values
        self.tk_inp = tk_inp.get_stoi()
        self.tk_out = tk_out.get_stoi()
        self.tokenizer = get_tokenizer('basic_english')
        self.max_len = max_len

    def __getitem__(self, i):
        # Input sequences
        encoder_seq = self.encoder_in[i]
        encoder_tokens = self.tokenizer(encoder_seq)
        encoder_indices = [self.tk_inp[token] for token in encoder_tokens]
        encoder_tensor = torch.tensor(encoder_indices)

        # Input encoder sequences
        decoder_in_seq = self.decoder_in[i]
        decoder_in_tokens = self.tokenizer(decoder_in_seq)
        # Special handling for <start> and <end> tokens
        decoder_in_indices = [self.tk_out['<start>']] + [self.tk_out[token] for token in decoder_in_tokens] + [self.tk_out['<end>']]
        decoder_in_tensor = torch.tensor(decoder_in_indices)

        # Input decoder sequences
        decoder_out_seq = self.decoder_out[i]
        decoder_tokens = self.tokenizer(decoder_out_seq)
        # Special handling for <start> and <end> tokens
        decoder_out_indices = [self.tk_out['<start>']] + [self.tk_out[token] for token in decoder_tokens] + [self.tk_out['<end>']]
        decoder_out_tensor = torch.tensor(decoder_out_indices)

        # Tokenizer padding
        encoder_tensor = F.pad(encoder_tensor, pad=(0, self.max_len - len(encoder_tensor)))
        decoder_in_tensor = F.pad(decoder_in_tensor, pad=(0, self.max_len - len(decoder_in_tensor)))
        decoder_out_tensor = F.pad(decoder_out_tensor, pad=(0, self.max_len - len(decoder_out_tensor)))

        return encoder_tensor, decoder_in_tensor, decoder_out_tensor

    def __len__(self):
        return len(self.encoder_in)


In [ ]:
train_data = conv_dataset(df_train, tk_inp, tk_out, 35)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

In [ ]:
my_dict = tk_inp.get_stoi()
my_dict["<start>i"]

4

In [ ]:
sample_index = 0
sample = train_data[sample_index]
tk_inp(encoder_seq)
sample

KeyError: ignored

#3)-Model Architecture

#3.1-Grammar error detection

##3.2-Grammar error correction


#4)-Training

#5)-Evaluation

#6)-Inference
